In [2]:
from music21 import *
import numpy as np
from midiutil import *
import csv

In [38]:
song = corpus.parse('bwv66.6')
song.show('text')

{0.0} <music21.metadata.Metadata object at 0x12e8444ccd0>
{0.0} <music21.stream.Part Soprano>
    {0.0} <music21.instrument.Instrument 'P1: Soprano: Instrument 1'>
    {0.0} <music21.stream.Measure 0 offset=0.0>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.key.Key of f# minor>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.note.Note C#>
        {0.5} <music21.note.Note B>
    {1.0} <music21.stream.Measure 1 offset=1.0>
        {0.0} <music21.note.Note A>
        {1.0} <music21.note.Note B>
        {2.0} <music21.note.Note C#>
        {3.0} <music21.note.Note E>
    {5.0} <music21.stream.Measure 2 offset=5.0>
        {0.0} <music21.note.Note C#>
        {1.0} <music21.note.Note B>
        {2.0} <music21.note.Note A>
        {3.0} <music21.note.Note C#>
    {9.0} <music21.stream.Measure 3 offset=9.0>
        {0.0} <music21.layout.SystemLayout>
        {0.0} <music21.note.Note A>
        {0.5} <music21.note.Note B>
        {1.0} <music21.note.Note

In [40]:
song.show('midi')

In [26]:
def MidiStringToInt(midstr):
    Notes = [["C"],["C#","Db"],["D"],["D#","Eb"],["E"],["F"],["F#","Gb"],["G"],["G#","Ab"],["A"],["A#","Bb"],["B"]]
    answer = 0
    i = 0
    #Note
    letter = midstr.split('-')[0].upper()
    for note in Notes:
        for form in note:
            if letter.upper() == form:
                answer = i
                break;
        i += 1
    #Octave
    answer += (int(midstr[-1]))*12
    return answer

In [67]:
partStream = song.getElementsByClass(stream.Part)
print(len(partStream))
#partStream.show('text')

#part.show('text')

parts = []
for p in partStream:
    part = []
    for n in p.flat.notes:
        note_with_num = MidiStringToInt(str(n.pitch.name) + "-" + str(n.pitch.octave))
        part.append([note_with_num, n.offset,  n.duration.quarterLength + n.offset])
    parts.append(part)
#print(parts_)
list_len = [len(p) for p in parts]
final_times = [p[-1][2] for p in parts]
max_final_time = max(final_times)
max_len = max(list_len)
parts_ = []
print([len(p) for p in parts])
for part in parts:
    num_notes_missing = max_len - len(part)
    if num_notes_missing > 1:
        notes_to_add = [[0, part[-1][2], part[-1][2]] for i in range(0, num_notes_missing - 1)]
        part = part + notes_to_add
    if num_notes_missing > 0:
        part = part + [[0, part[-1][2], max_final_time]]
    #print(part)
    parts_.append(part)
all_notes = np.array(parts_)
all_notes

4
[37, 42, 45, 41]


array([[[61. ,  0. ,  0.5],
        [59. ,  0.5,  1. ],
        [57. ,  1. ,  2. ],
        [59. ,  2. ,  3. ],
        [61. ,  3. ,  4. ],
        [64. ,  4. ,  5. ],
        [61. ,  5. ,  6. ],
        [59. ,  6. ,  7. ],
        [57. ,  7. ,  8. ],
        [61. ,  8. ,  9. ],
        [57. ,  9. ,  9.5],
        [59. ,  9.5, 10. ],
        [56. , 10. , 11. ],
        [54. , 11. , 12. ],
        [57. , 12. , 13. ],
        [59. , 13. , 14. ],
        [59. , 14. , 15. ],
        [54. , 15. , 16. ],
        [52. , 16. , 17. ],
        [57. , 17. , 18. ],
        [59. , 18. , 19. ],
        [61. , 19. , 20. ],
        [61. , 20. , 21. ],
        [57. , 21. , 22. ],
        [59. , 22. , 23. ],
        [61. , 23. , 24. ],
        [57. , 24. , 25. ],
        [56. , 25. , 26. ],
        [54. , 26. , 27. ],
        [56. , 27. , 29. ],
        [54. , 29. , 31. ],
        [54. , 31. , 32. ],
        [54. , 32. , 33. ],
        [54. , 33. , 34. ],
        [54. , 34. , 34.5],
        [48. , 34.5,

In [69]:
#degrees  = [60, 62, 64, 65, 67, 69, 71, 72]  # MIDI note number
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 60   # In BPM
volume   = 100  # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1)  # One track
MyMIDI.addTempo(track, time, tempo)

for ch, part_ in enumerate(all_notes):
    for note in part_:
        if int(note[0]) != 0:
            MyMIDI.addNote(track, ch, int(note[0]), note[1], note[2], volume)

with open("midi-to-vector-take-2.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)